In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
confirmed = pd.read_csv('time_series_covid19_confirmed_US.csv')
deaths = pd.read_csv('time_series_covid19_deaths_US.csv')


In [3]:
confirmed.shape

(3342, 555)

In [4]:
deaths.shape

(3342, 556)

In [23]:
confirmed.drop(['UID', 'iso2', 'iso3', 'code3', 'Lat', 'Long_', 'Country_Region', 'Combined_Key'], axis=1, inplace=True)
deaths.drop(['UID', 'iso2', 'iso3', 'code3', 'Lat', 'Long_', 'Country_Region', 'Combined_Key'], axis=1, inplace=True)

In [24]:
confirmed.head()
deaths.head()

,FIPS,Admin2,Province_State,Population,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/9/21,7/10/21,7/11/21,7/12/21,7/13/21,7/14/21,7/15/21,7/16/21,7/17/21,7/18/21
0,1001.0,Autauga,Alabama,55869,0,0,0,0,0,0,...,113,113,113,113,113,113,113,113,113,113
1,1003.0,Baldwin,Alabama,223234,0,0,0,0,0,0,...,317,318,318,318,318,321,323,323,323,323
2,1005.0,Barbour,Alabama,24686,0,0,0,0,0,0,...,60,60,60,60,60,60,60,60,60,60
3,1007.0,Bibb,Alabama,22394,0,0,0,0,0,0,...,65,65,65,65,65,65,65,65,65,65
4,1009.0,Blount,Alabama,57826,0,0,0,0,0,0,...,139,139,139,139,139,139,139,139,139,139


In [33]:
confirmed.rename(columns={'Admin2':'County',
                          'Province_State':'State',
                          'Confirmed':'Cases'}, inplace=True)
deaths.rename(columns={'Admin2':'County',
                          'Province_State':'State',
                       'Confirmed':'Deaths'}, inplace=True)

In [34]:
confirmed_long = confirmed.melt(
    id_vars=['FIPS', 'County', 'State'], 
    var_name='Date', 
    value_name='Cases'
)

deaths_long = deaths.melt(
    id_vars=['FIPS', 'County', 'State', 'Population'], 
    var_name='Date', 
    value_name='Deaths'
)

In [35]:
confirmed_long.tail()
deaths_long.tail()

#print(confirmed_long.shape, deaths_long.shape)

,FIPS,County,State,Population,Date,Deaths
1818043,56039.0,Teton,Wyoming,23464,7/18/21,11
1818044,56041.0,Uinta,Wyoming,20226,7/18/21,13
1818045,90056.0,Unassigned,Wyoming,0,7/18/21,0
1818046,56043.0,Washakie,Wyoming,7805,7/18/21,26
1818047,56045.0,Weston,Wyoming,6927,7/18/21,6


In [36]:
covid_long = confirmed_long.merge(
  right=deaths_long, 
  how='left',
  on=['State', 'County', 'Date']
)

In [42]:
covid_long.head()

,FIPS_x,County,State,Date,Cases,FIPS_y,Population,Deaths
0,1001.0,Autauga,Alabama,1/22/20,0,1001.0,55869,0
1,1003.0,Baldwin,Alabama,1/22/20,0,1003.0,223234,0
2,1005.0,Barbour,Alabama,1/22/20,0,1005.0,24686,0
3,1007.0,Bibb,Alabama,1/22/20,0,1007.0,22394,0
4,1009.0,Blount,Alabama,1/22/20,0,1009.0,57826,0


In [57]:
covid_long[covid_long['County'].isnull()].head()

,FIPS_x,County,State,Date,Cases,FIPS_y,Population,Deaths
102,60.0,NaN,American Samoa,1/22/20,0,60.0,55641,0
338,88888.0,NaN,Diamond Princess,1/22/20,0,88888.0,0,0
572,99999.0,NaN,Grand Princess,1/22/20,0,99999.0,0,0
573,66.0,NaN,Guam,1/22/20,0,66.0,164229,0
2123,69.0,NaN,Northern Mariana Islands,1/22/20,0,69.0,55144,0


In [47]:
null_states

array(['American Samoa', 'Diamond Princess', 'Grand Princess', 'Guam',
       'Northern Mariana Islands', 'Virgin Islands'], dtype=object)

In [58]:
covid_long = covid_long[~covid_long['State'].isin(null_states)]

In [60]:
covid_long.to_csv('JHU_cases_deaths.csv', index=False)